In [1]:
import pandas as pd
import os
import numpy as np
import epiweeks
import datetime
import requests
import zipfile
import csv
from contextlib import ExitStack
import progressbar as pbar
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
today = datetime.date.today().strftime("%d-%m-%Y")
data_dir_path = f'data/World/{today}'
if not os.path.exists(f'{data_dir_path}'):
        os.mkdir(f'{data_dir_path}')

In [3]:
# GERMANY
country = 'Germany'
if not os.path.exists(f'{data_dir_path}/{country}'):
    os.mkdir(f'{data_dir_path}/{country}')

cases_csv = 'https://raw.githubusercontent.com/KITmetricslab/covid19-forecast-hub-de/master/data-truth/RKI/by_age/truth_RKI-Incident%20Cases%20by%20Age_Germany.csv'
cases = pd.read_csv(cases_csv)
cases['age_group'].replace(dict([['A00-A04', '00-04'], ['A05-A14', '05-14'], ['A15-A34', '15-34'], ['A35-A59', '35-59'], ['A60-A79', '60-79'], ['A80+', '80+']]), inplace=True)
cases = cases.drop(cases.loc[cases['age_group']=='unbekannt'].index)
cases = cases.drop(columns='location_name').rename(columns={'age_group': 'Age group', 'location': 'location_name'})
cases['location_name'] = cases['location_name'].apply(lambda x: 'DE'+x[2:])
cases.to_csv(f'{data_dir_path}/{country}/covid_cases.csv')

deaths_csv = 'https://raw.githubusercontent.com/KITmetricslab/covid19-forecast-hub-de/master/data-truth/RKI/by_age/truth_RKI-Incident%20Deaths%20by%20Age_Germany.csv'
deaths = pd.read_csv(deaths_csv)
deaths.loc[deaths['location_name'] == 'Free State of Thuringia', 'location_name'] = 'Free State of Thüringia'
deaths['age_group'].replace(dict([['A00-A04', '00-04'], ['A05-A14', '05-14'], ['A15-A34', '15-34'], ['A35-A59', '35-59'], ['A60-A79', '60-79'], ['A80+', '80+']]), inplace=True)
deaths = deaths.drop(deaths.loc[deaths['age_group']=='unbekannt'].index)
deaths = deaths.drop(columns='location_name').rename(columns={'age_group': 'Age group', 'location': 'location_name'})
deaths['location_name'] = deaths['location_name'].apply(lambda x: 'DE'+x[2:])
deaths.to_csv(f'{data_dir_path}/{country}/covid_deaths.csv')